[ 💡  ] 에 답을 넣어주세요. 

In [ ]:
# Attention mechanism 

Seq2Seq 모델의 문제점 :
Seq2Seq 모델은 Encoder에서 입력 시퀀스에 대한 특성을 [💡     ] 에 압축하여 Decoder로 전달 한다.  
하나의 고정된 크기의 vector에 모든 입력 시퀀스의 정보를 넣다보니 [ 💡   ] 이 발생한다. 
seq2seq는 encoder의 마지막 hidden state를 context로 받은 뒤 그것을 이용해 모든 출력 단어들을 생성하므로 그 중요도에 대한 반영이 안된다.



## 기존의 Seq2seq의 특징 
 - 짧은 문장의 경우 : "오늘 저녁에 뭐할거야?" 
 - Encoder는 질문 하나에 벡터 하나씩 
 - Decoder는 이 벡터 하나씩에 대해 답을 만듬 ("친구 만나러 가요")

### 여기서 문제는? 
- 질문이 길수록 벡터가 너무 많아져서 Decoder가 답을 만들기가 빡세다. 

### 그래서 등장한 Attention / Attention의 필요성
- 벡터 하나에 모든 걸 다 넣지 말고, 필요할 때 마다 주어진 문장(문서)를 다시 참고하자. 
- 출력하는 단어마다(매 시점(time step)마다) 입력된 문장, 문서(context vector)에서 어디를 봐야할지(집중(attention)) 설정하는 것! 

# Data Loading

In [ ]:
import os
import pandas as pd
import requests

os.makedirs("data", exist_ok=True)
url = "https://raw.githubusercontent.com/songys/Chatbot_data/refs/heads/master/ChatbotData.csv"
res = requests.get(url)
if res.status_code == 200:
    with open("data/chatbot_data.csv", "wt", encoding="utf-8") as fw:
        fw.write(res.text)
else:
    print(f"불러오지 못함: {url}")

In [ ]:
df = pd.read_csv('data/chatbot_data.csv')
df.drop(columns='label', inplace=True)
df.head()

# 토큰화

In [ ]:
question_texts = df['Q'] # 질문 모음 
answer_texts = df['A']   # 답변 모음
all_texts = list(question_texts + " "+answer_texts) # Q + A : vocab 생성
# all_texts에 대해 토큰화를 진행하면, 모델이 학습할 수 있는 형태로 바뀌게 된다.
len(question_texts), len(answer_texts), len(all_texts)

## Tokenizer 학습

1. BPE 기반 토크나이저 만들고
2. 공백 단위로 단어를 자르게 하고
3. 학습 조건을 설정한 뒤
4. 실제로 학습시킨다 (내 문장들을 넣어서)

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 10000  # 뜻 : <최대 단어 사전 크기> 10,000개로 제한
min_frequency = 5   # 뜻 : <최소 5번 이상 > 단어만 포함

tokenizer = Tokenizer(BPE(<unk_token="[UNK]")>) # 모르는 단어 일 때 [UNK]를 넣는다. 
tokenizer.pre_tokenizer = [💡  ] # 공백기준으로 문장을 쪼개겠다. 
trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=min_frequency,
    continuing_subword_prefix='##',
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"] 
    # [SOS]: <문장의 시작을 의미>하는 토큰. [EOS]: <문장이 끝난 것을 의미>하는 토큰.
)

tokenizer.train_from_iterator(all_texts, trainer=trainer)

In [ ]:
print("총 어휘수:", tokenizer.get_vocab_size())

## 저장

In [ ]:
dir_path = "saved_model/chatbot_attn"
os.makedirs(dir_path, exist_ok=True)
vocab_path = os.path.join(dir_path, "chatbot_attn_bpe.json")
tokenizer.save(vocab_path)

# Dataset 생성
- 한문장 단위로 학습시킬 것이므로 DataLoader를 생성하지 않고 Dataset에서 index로 조회한 질문-답변을 학습시킨다.

In [ ]:
여기서 질문!
Q: 여기서 data loader를 생성하지않고 dataset에서 index로 조회한 질문 답변을 학습시킨다는 의미는 무엇일까요 ?

A: 직접 인덱스로 조회 - [ 💡  ] 
    데이터 로더 사용 - [ 💡  ]

# 실제 학습에서는 data loader로 배치 단위로 처리하는것이 일반적이라고 합니다.! 

In [ ]:
import random
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "mps" if torch.mps.is_available() else "cpu"
device

### Dataset 클래스 정의

In [ ]:
class ChatbotDataset(Dataset):

    """
    Attribute
        max_length
        tokenizer: Tokenizer
        vocab_size: int - Tokenizer에 등록된 총 어휘수
        SOS: int - [SOS] 문장의 시작 토큰 id
        EOS: int = [EOS] 문장의 끝 토큰 id
        question_squences: list - 모든 질문 str을 token_id_list(token sequence) 로 변환하여 저장한 list 
        answser_sequences: list - 모든 답변 str을 token_id_list(token sequence) 로 변환하여 저장한 list.
    """
    def __init__(self, question_texts, answer_texts, tokenizer, min_length=2, max_length=20):
        """
        question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
        answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1",   "답2",   ...]
        tokenizer: Tokenizer
        min_length=2: int - 최소 토큰 개수. 질문과 답변의 token수가 min_length 이상인 것만 학습한다.
        max_length=20:int 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        """
        self.min_length = min_length
        self.max_length = max_length
        self.tokenizer = tokenizer
        
        self.vocab_size = tokenizer.get_vocab_size()
        self.SOS = self.tokenizer.token_to_id('[SOS]')
        self.EOS = self.tokenizer.token_to_id('[EOS]')

        self.question_sequences = []
        self.answer_sequences = []
        for q, a in zip(question_texts, answer_texts):
            q_token = [💡  ](q) # 각 문장을 숫자리스트로 바꿈 
            a_token = [💡  ](a) # 각 문장을 숫자리스트로 바꿈 
            if len(q_token) > min_length and len(a_token) > min_length:
                self.question_sequences.[💡  ](q_token)
                self.answer_sequences.[💡  ] (a_token) # 길이가 충분한 질문, 답변만 최종 학습용 리스트에 저장 

    def __add_special_tokens(self, token_sequence):
        """
        질문/답변 토큰 리스트 맨 뒤에 문장의 끝을 표시하는 [EOS] 토큰 추가. 
        [EOS] Token을 붙이고 max_length 보다 토큰수가 많으면 안된다.
        Args:
            token_sequence (list[str]) - EOS 토큰을 추가할 문서 token sequence
        """
        token_id_list = token_sequence [💡 ]# 문장 끝을 표시하는 [EOS] 를 추가하기 위해 마지막 하나 잘라낸다.         token_id_list.append(self.EOS)

        return token_id_list

    def __process_sequence(self, text):
        """
        한 문장 string을 받아서 encoding 한 뒤 [EOS] token을 추가한 token_id 리스트(list)를 생성 해서 반환한다.
        Args:
            text (str) - token id 리스트로 변환할 대상 String.
        """
        encode = self.tokenizer.encode(text)
        token_ids = self.__add_special_tokens(encode.ids)
        return token_ids
    
    def __len__(self):
        return len(self.question_sequences)

    def __getitem__(self, index):   
        # embedding 입력 -> int 64
        # unsqueeze(1) - [1, 2, 3, 4] -> [[1], [2], [3], [4]] # 차원을 추가하는 함수 
        q = torch.tensor(self.question_sequences[index], dtype=torch.int64).[💡  ] # 차원을 추가하는 함수 
        a = torch.tensor(self.answer_sequences[index], dtype=torch.int64).[💡   ] # 차원을 추가하는 함수 
        return q, a

### Dataset 객체 생성

In [ ]:
MAX_LENGTH = 20
MIN_LENGTH = 2
dataset = ChatbotDataset(question_texts, answer_texts, tokenizer, MIN_LENGTH, MAX_LENGTH)
print(len(dataset))

# 모델

# Encoder
GRU를 통해 문맥 정보를 압축해주는 역할

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self, num_vocabs, hidden_size, embedding_dim, num_layers):
        """
        Args:
            num_vocabs: int - 총 어휘수 
            hidden_size: int - GRU의 hidden size
            embedding_dim: int - Embedding vector의 차원수 
            num_layers: int - GRU의 layer수
        """
        super().__init__()
        self.num_vocabs = num_vocabs
        self.hidden_size = hidden_size

        # 임베딩 레이어, 단어를 벡터로 변환
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)

        # GRU 생성, 단어 순서에 따라 문맥을 이해 
        self.gru = nn.GRU(
            input_size=embedding_dim, 
            hidden_size=hidden_size, 
            num_layers=num_layers
            )


    def [ 💡 ]:
        """
        질문의 token한개의 토큰 id를 입력받아 hidden state를 출력
        
        Args:
            x: 한개 토큰. shape-[1]
            hidden: hidden state (이전 처리결과). shape: [1, 1, hidden_size]
        Returns
            tuple: (output, hidden) - output: [1, 1, hidden_size],  hidden: [1, 1, hidden_size]
        """
        # x shape : [batch: 1, 1]
        embedded = self.embedding(x).unsqueeze(0) # (1: batch, embedding_dim) -> (1: batch, 1: seq_len, embedding_dim)
        out, hidden = self.gru(embedded, hidden)

        return out, hidden 
    
    
    def [  💡 ]: #처음 timestep에서 입력할 hidden_state.
        """
        (왜냐면 첨 스텝에는 넣어야할 hidden state 가 없기 때문에 직접 만들어서 넣어주는 것)
        값: < 0 >
        shape: (Bidirectional(1) x number of layers(1), batch_size: 1, hidden_size) 
        """
        return torch.zeros(1, 1, self.hidden_size, device=device)
    

## Attention 적용 Decoder



- Attention은 Decoder 네트워크가 순차적으로 다음 단어를 생성하는 자기 출력의 모든 단계에서 인코더 출력 중 연관있는 부분에 **집중(attention)** 할 수 있게 한다. 
- 다양한 어텐션 기법중에 **Luong attention** 방법은 다음과 같다.


In [ ]:
### [💡  ]
- Decoder가 현재 timestep의 단어(token)을 생성할 때 Encoder의 output 들 중 어떤 단어에 좀더 집중해야 하는지 계산하기 위한 가중치값.


In [ ]:
### [💡  ]
- Decoder에서 현재 timestep의 단어를 추출할 때 사용할 Context Vector.
    - Encoder의 output 들에 Attention Weight를 곱한다.
    - Attention Value는 Decoder에서 단어를 생성할 때 encoder output의 어떤 단어에 더 집중하고 덜 집중할지를 가지는 값이다.


In [ ]:
### Feature Extraction
- Decoder의 embedding vector와 Attention Value 를 합쳐 RNN(GRU)의 입력을 만든다.
    - **단어를 생성하기 위해 이전 timestep에서 추론한 단어 현재 timestep의 input 와 Encoder output에 attention이 적용된 값 이 둘을 합쳐 입력한다.
    - 이 값을 [💡  ]를 이용해 RNN input_size에 맞춰 준다. (어떻게 input_size에 맞출지도 학습시키기 위해 Linear Layer이용)


### 단어 예측(생성)
- RNN에서 찾은 Feature를 총 단어개수의 units을 출력하는 Linear에 입력해 **다음 단어를 추론한다.**
- 추론한 단어는 다음 timestep의 입력($X_t$)으로 RNN의 hidden은 다음 timestep 의 hidden state ($h_{t-1}$) 로 입력된다.


In [ ]:
class AttentionDecoder(nn.Module):

    def __init__(self, num_vocabs, hidden_size, embedding_dim, dropout_p, max_length):
        # num_vocabs : 총 어휘수
        super().__init__()
        self.num_vocabs = num_vocabs
        self.hidden_size = hidden_size
        self.max_length = max_length

        # embedding layer
        self.embedding = nn.Embedding(num_vocabs, embedding_dim)

        # attention weight를 계산하는 Linear
        # 이전 단어의 hidden state(prev_hidden)와 현재 단어의 embedding vector에
        #   가중합을 계산해서 attention weight를 계산
        # in_features: hidden_size + embedding_dim 
        # out_features: Encoder의 hidden_state의 개수 (max_length)
        self.attn = nn.Linear(hidden_size+embedding_dim, max_length)

        # 가정 : hidden_size = 200, max_length (토큰 수) = 20, 
        # attention value: attention- weight @ encoder의 hidden state들 (out)
        # shape: 1 x 20 @ 20 x 200 = 1 x 200 

        # 현재 단어 embedding vector + attention balue를 입력받아 가중합을 계산해서 
            # GRU(RNN)에 입력할 입력값을 계산 
            # in_features : embedding_dim + encoder의 hidden_size
        self.attn_combine = nn.Linear(embedding_dim+hidden_size, hidden_size)

        self.dropout = nn.Dropout(dropout_p)

        # GRU
        self.gru = nn.GRU(hidden_size, hidden_size)

        # 분류기 
        self.classifier = nn.Linear(hidden_size, num_vocabs)



    def forward(self, x, hidden, encoder_outputs):
        """
        Parameter
            x: 현재 timestep의 입력 토큰(단어) id
            hidden: 이전 timestep 처리결과 hidden state
            encoder_outputs: Encoder output들. 
        Return
            tupe: (output, hidden, attention_weight)
                output: 단어별 다음 단어일 확률.  shape: [vocab_size]
                hidden: hidden_state. shape: [1, 1, hidden_size]
                atttention_weight: Encoder output 중 어느 단어에 집중해야하는 지 가중치값. shape: [1, max_length]
        
        현재 timestep 입력과 이전 timestep 처리결과를 기준으로 encoder_output와 계산해서  encoder_output에서 집중(attention)해야할 attention value를 계산한다.
        attention value와 현재 timestep 입력을 기준으로 단어를 추론(생성) 한다.
        """
        
        #  embedding 
        embedding = self.embedding(x).unsqueeze(0) # [1:batch] -> [1:batch, 1:seq_len]
        embedding = self.dropout(embedding)

        # attetion weight 계산 
        attn_in = torch.[💡  ]((embedding[0], hidden[0]),dim=1 ) #현재 단어 임베딩, 이전 히든 이어붙여 각 단어별 중요도 점수 생성
        # attn_in shape : [1, embedding_dim+hidden_size]
        attn_score = self.attn(attn_in) # logit
        # shape : < [1, max_length]> 
        # attn_score shape: 1 x embeddign_dim+hidden_size @ embedding_dim+hidden_size x max_length

        
        attn_weight = nn.Softmax[💡    ](attn_score) # 전체 문장에서 어떤 단어에 집중할지 확률로 변환
        # 마지막 차원을 기준으로 softmax 적용해라 - **문장 안의 단어 위치별 집중 정도(확률)**를 구하는 데 쓰입니다.


        


In [ ]:
        # attetion value 계산 (attn_applied) 
        #  attn_weight @ encoder_hiddenstate
        ##  1 x max_length  @  max_length x hidden_size 

        # torch.bmm() - batch-wise matrix multiplication(배치단위 행렬곱) - 3차원 텐서만 받음! 
        ##  3차원 배열을 받아서 1, 2 축 기준으로 행렬곱 계산. 
        ### (5, 2, 3) @ (5, 3, 5) -> 2 x 3 @ 3 x 5 5개를 행렬곱 => (5, 2, 5)
        attn_value = torch.bmm(
            attn_weight.[💡   ], # (1, 1, max_length) # 배치차원 추가
            encoder_outputs.[💡    ]>, # (1, max_length, hidden_size)
        )
        # attn_value 결과: attn_weight @ encoder_outputs (1:batch, 1, hidden_size)
        # 각 단어 벡터에 집중정도 attn_weight를 곱해서 문장의 대표 의미 attention value를 뽑아냄 


### torch.bmm()
- 요약 : attention 가중치를 각 encoder hidden에 곱해 **중요한 단어에 집중한 문맥 벡터(attn_value)**를 계산하는 연산

In [ ]:
        # attn_combine: gru의 input 값을 생성 
        ## attn_value + embedding(concat) => Linear => ReLU


        attn_combine_in = torch.concat([
            [ 💡   ]  , ### 지금까지 입력 문장을 압축한 문맥 벡터(1 x hidden_size)
            [ 💡    ]       ### 현재 디코더에 입력된 단어의 임베딩 벡터 (1 x embedding_dim)
        ], dim = 1)
        gru_in = self.attn_combine(attn_combine_in) # 합쳐진 벡터를 Linear 레이어에 통과시켜서 hidden size로 변환 # 출력 (1, hidden_size)
        gru_in = gru_in.<unsqueeze(0)> ## # 여기서 왜함? ****
        gru_in = nn.ReLU()(gru_in) # 비선형성 추가 

        # gru에 입력해서 다음 단어를 찾기 위한 hidden state(feature)를 계산.
        out, hidden_state = self.gru(gru_in, hidden) # (seq, batch, hidden_size) (1, 1, hidden_size) # hidden_size가 만약 200개면, [[[0, 1, 2, 3, 4 .... 199]]]

        # classification 에 out 을 입력해서 다음 단어를 예측
        last_out = self.classifier(out[0]) # 분류기는 2차원 자료구조, linear할 떄 생각하면... [0] 면, [[0, 1, 2, 3, 4 .... 199]] 괄호하나 지운것. 
        # gru에서 빠져나온 마지막 단어에 대한 확률 
        # last_out shape: [  [1, num_vocabs]  ] 0번째 인덱스로 조회해야 우리가 알고싶은 값이 나온다. 
        # decode 에서 빠져 나온 값을 다음 gru, 다음 gru에서 쓰도록 0 인덱스에서 사이즈를 맞춰줌 


        return last_out[0], hidden_state, attn_weight
                #  last_out[0]의 값은 [0, 1, 2, 3, 4 .... 199] 괄호하나 더 지운것. 
                #  last_out : 실제 vocab 사이즈만큼의 확률 리스트 


*** 
GRU는 입력 shape을 **(seq_len, batch_size, hidden_size)**로 받습니다.

현재 gru_in의 shape은 (1, hidden_size)로 2D입니다.

→ unsqueeze(0)으로 맨 앞에 sequence 길이 차원을 추가해서 (1, 1, hidden_size)로 만들어줘야 합니다.

###  Encoder와 AttentionDecoder가 잘 작동하는지 확인하기 위해 dummy 데이터를 통해 모델을 직접 생성하고, 구조를 시험해보는 과정

In [ ]:
# Encoder/ Decoder 를 dummy_data로 확인
dummy_encoder = Encoder(
    num_vocabs=tokenizer.get_vocab_size(), # Tokenizer가 학습한 전체 단어 개수
    hidden_size=256,                       # RNN의 hidden state
    embedding_dim=200,                     # 단어를 표현할 embedding vector의 차원
    num_layers=1
)

dummy_encoder = dummy_encoder.to(device)

dummy_decoder = AttentionDecoder(
    num_vocabs=tokenizer.get_vocab_size(),
    hidden_size=256,
    embedding_dim=200,
    [💡   ] =0.3,                          # 훈련 중 일부 노드를 랜덤으로 꺼서 과대적합 방지
    [💡   ] =20                             # 입력 시퀀스의 최대 길이로 attention weight의 크기 결정에 사용됨
)

dummy_decoder = dummy_decoder.to(device)

In [ ]:
x, y = dataset[0] # 첫번째 (Q, A)
x, y = x.to(device), y.to(device)


# 첫번째 질문의 첫번째 토큰을 입력
encoder_out, encoder_hidden = dummy_encoder([💡    ], dummy_encoder.init_hidden(device))
# x[0] 토큰은 ? [SOS] 토큰이다! 

encoder_out.shape, encoder_hidden.shape


In [ ]:
# 첫번째 질문의 첫번째 토큰을 입력. y[0]
encoder_outputs = torch.randn(20, 256, device=device) # 20: seq_len, 256: hidden_size
next_token, hidden_state, attn_weight = dummy_decoder([💡    ], encoder_out, encoder_outputs) # 현재 단어(y[0])와 이전 hidden state를 입력받음
# 결과적으로 다음 단어가 무엇인지 확률 분포(next_token)로 출력

next_token        # 예측한 다음 단어의 확률 분포 (shape: [vocab_size])
hidden_state      # 업데이트된 hidden state (shape: [1, 1, hidden_size])
attn_weight       # 어떤 encoder 단어에 집중했는지 (shape: [1, seq_len])

In [ ]:
print(next_token.shape)
print(next_token.argmax(-1), tokenizer.id_to_token(next_token.argmax(-1).item()))
print(hidden_state.shape)

In [ ]:
print(attn_weight.shape)
attn_weight